## Robo Advisor for Retirement plans
## Lambda function used for Robo advisor created in Amaxon Lex

In [ ]:
### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Functionality Helper Functions ###
# Required Libraries: This section contains all the necessary libraries to code the business logic on the lambda functions. 
# Although AWS Lambda supports Python, the AWS Lambda's runtime doesn't support some common packages, such as pandas, numpy or requests,
# so alternative packages should be used or installed.

# Functionality Helper Functions: These functions implement business logic and data validation. In this robo advisor, there is 2 helper functions.
# parse_int(): This function securely parses a non-numeric value to float.
# build_validation_result(): This function defines an internal validation message structured as a Python dictionary.
# data_validation(): This function validates the data provided by the user across the intent's dialogue on Amazon Lex according to the business logic. 
# data_validation rules applied: (1) the user should be greater than zero and less than 65 years old (2) the investment amount must be equal to or greater than 5000.
# investment_recommendation(): This function validates the risk level appetite selected by the user and responds with an investment recommendation.  

def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }

def data_validation(age, investment_amount, intent_request):
        # Age should be greater than 0 and less than 65 years old
        if age is not None:
            age = parse_int(age)
            if age >= 65:
                return build_validation_result(
                    False,
                    "age",
                    "You should be at least 65 to use this service, please provide a different age."
                    )
            if age <= 0:
                return build_validation_result(
                    False,
                    "age",
                    "You should be older than 0 to use this service, please provide a different age."
                    )
            
        # Investment amount should be equal to or greater than 5000
        if investment_amount is not None:
            investment_amount = parse_int(investment_amount)    
       
            if investment_amount < 5000:
      
                return build_validation_result(
                    False,
                    "investmentAmount",
                    "The initial investment needs to be greater than 5000, please provide a valid amount."
                    )
        return build_validation_result(True, None, None)
        
def investment_recommendation(risk_level):
    # Investment recommendation proposed based on risk level selected
    if risk_level == "None":
        initial_recommendation = "100% bonds (AGG), 0% equitties (SPY)"
    elif risk_level == "Very Low":
        initial_recommendation = "80% bonds (AGG), 20% equitties (SPY)"
    elif risk_level == "Low":
        initial_recommendation = "60% bonds (AGG), 40% equitties (SPY)"
    elif risk_level == "Medium":
        initial_recommendation = "40% bonds (AGG), 60% equitties (SPY)"
    elif risk_level == "High":
        initial_recommendation = "20% bonds (AGG), 80% equitties (SPY)"
    elif risk_level == "Very High":
        initial_recommendation = "0% bonds (AGG), 100% equitties (SPY)"
        
    return initial_recommendation

### Dialog Actions Helper Functions ###
# Dialog Actions Helper Functions: These functions handle the input and response events data from the conversation 
# between Amazon Lex and AWS Lambda. The get_slots() function fetches all the slots and their values from the current intent. 
# The elicit_slot(), delegate() and close() functions, construct response messages structured as a valid JSON Lex event. 
# The full structure of event data that Amazon Lex exchanges with a Lambda function can be reviewed here.

def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }


def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }
    return response


### Intents Handlers ###

#Intents Handlers: The core business logic is coded into an intent handler. 
#An intent handler is a function that implements the functionality that is willing to fulfill the user's intent.
#The recommend_portfolio() function contains all the logic to validate the user's input stored in the slots using the data_validation() and  
#investment_recommendation helper functions; along with the conversation between the user and the bot, if any of the slots have invalid data, an elicitSlot 
#dialogue is returned to request the data again to the user, otherwise a delegate() dialogue is returned to direct Lex to choose the 
#next course of action according to the bot's configuration.

#The data_validation() function and initial_recommendation() is done, and the close() dialogue function is called to return a Fulfilled event message to Lex.

def recommend_portfolio(intent_request):
    """
    Performs dialog management and fulfillment for recommending a portfolio.
    """
    first_name = get_slots(intent_request)["firstName"]
    age = get_slots(intent_request)["age"]
    investment_amount = get_slots(intent_request)["investmentAmount"]
    risk_level = get_slots(intent_request)["riskLevel"]
    source = intent_request["invocationSource"]

    if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        slots = get_slots(intent_request)
        
        # AGE greater than 0 and less than 65
        validation_result = data_validation(age, investment_amount, intent_request)

        # If the data provided by the user is not valid,
        # the elicitSlot dialog action is used to re-prompt for the first violation detected.
        if not validation_result["isValid"]:
            slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot

            # Returns an elicitSlot dialog to request new data for the invalid slot
            return elicit_slot(
                intent_request["sessionAttributes"],
                intent_request["currentIntent"]["name"],
                slots,
                validation_result["violatedSlot"],
                validation_result["message"],
            )
        # Fetch current session attibutes
        output_session_attributes = intent_request["sessionAttributes"]

            
        return delegate(output_session_attributes, get_slots(intent_request))

    # Get the initial investment recommendation
    initial_recommendation = investment_recommendation(risk_level)


    # Return a message with the initial recommendation based on the risk level.
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """{} thank you for your information;
            based on the risk level you defined, my recommendation is to choose an investment portfolio with {}
            """.format(
                first_name, initial_recommendation
            ),
        },
    )


### Intents Dispatcher ###
# An Amazon Lex bot can have one or more intents. The purpose of the dispatch() function is to validate that the current intent is valid, 
# and to dispatch the intent to the corresponding intent handler. There is one intent, thus one intent handler which calls the recommend_portfolio() function.


def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "RecommendPortfolio":
        return recommend_portfolio(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)
